# Phase 2: NPS指標の算出とデータ集計
このステップでは、生成されたダミーデータを読み込み、NPS（ネット・プロモーター・スコア）を算出します。
セグメントごとのNPSを可視化することで、自社の強みと弱みを特定可能な形に加工します。

### 本フェーズの目標
- NPSスコアから「推奨者」「中立者」「批判者」の割合を算出。
- 各属性（会社、チャネル、年代）ごとのNPSを計算。
- Power BIでの分析に適した「集計済みデータ」の書き出し。

In [1]:
import pandas as pd
import numpy as np

# データの読み込み
df = pd.read_csv("../data/dummy_nps_data.csv")

# データ型の確認
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  2000 non-null   object
 1   company      2000 non-null   object
 2   channel      2000 non-null   object
 3   age_group    2000 non-null   int64 
 4   tenure_year  2000 non-null   int64 
 5   nps_score    2000 non-null   int64 
 6   nps_segment  2000 non-null   object
dtypes: int64(3), object(4)
memory usage: 109.5+ KB
None


,customer_id,company,channel,age_group,tenure_year,nps_score,nps_segment
0,CID0001,C社,営業職員,50,9,7,中立者
1,CID0002,B社,営業職員,40,17,8,中立者
2,CID0003,A社(自社),営業職員,30,10,4,批判者
3,CID0004,B社,ネット,20,4,7,中立者
4,CID0005,B社,営業職員,30,7,7,中立者


## 1. NPS集計関数の定義
NPSは以下の数式で定義されます。
$$NPS = (\frac{推奨者の数}{回答者合計} - \frac{批判者の数}{回答者合計}) \times 100$$
この値を算出するカスタム関数を作成します。

In [2]:
def calculate_nps_score(series):
    """
    PandasのSeries（nps_segment列）を受け取り、NPSスコアを返す関数
    """
    total_count = len(series)
    counts = series.value_counts()
    
    # 推奨者・批判者の割合を算出（存在しない場合は0）
    promoters = counts.get('推奨者', 0)
    detractors = counts.get('批判者', 0)
    
    nps = ((promoters - detractors) / total_count) * 100
    return round(nps, 1)

# 全体NPSの確認
overall_nps = calculate_nps_score(df['nps_segment'])
print(f"全体NPS: {overall_nps}")

全体NPS: -41.1


## 2. セグメント別集計
「会社 × チャネル」や「会社 × 年代」などの切り口でNPSを算出し、比較を行います。

In [3]:
# 会社別・チャネル別NPS
nps_by_channel = df.groupby(['company', 'channel'])['nps_segment'].apply(calculate_nps_score).unstack()

# 会社別・年代別NPS
nps_by_age = df.groupby(['company', 'age_group'])['nps_segment'].apply(calculate_nps_score).unstack()

print("--- チャネル別NPS比較 ---")
print(nps_by_channel)
print("\n--- 年代別NPS比較 ---")
print(nps_by_age)

--- チャネル別NPS比較 ---
channel   ネット  営業職員
company            
A社(自社)  -57.8 -26.7
B社       11.2 -50.6
C社      -62.8 -55.6

--- 年代別NPS比較 ---
age_group    20    30    40    50    60    70    80
company                                            
A社(自社)    -63.2 -64.9 -59.0 -48.0  -2.8  -9.8 -14.3
B社        -16.7 -25.5 -29.9 -20.8 -34.8 -22.4 -43.9
C社        -62.5 -53.5 -52.3 -61.0 -57.9 -63.6 -71.4


## 3. Power BI向け最終加工
Power BIで「回答者の内訳（％）」と「NPS」を同時に表現できるよう、集計済みテーブルを保存します。

In [4]:
# 会社・チャネル・年代ごとの回答者数とNPSを一つのテーブルにまとめる
summary_df = df.groupby(['company', 'channel', 'age_group']).agg(
    respondents=('customer_id', 'count'),
    avg_score=('nps_score', 'mean'),
    nps=('nps_segment', calculate_nps_score)
).reset_index()

# 集計結果の保存
summary_df.to_csv("../data/nps_summary.csv", index=False, encoding="utf-8-sig")
print("Power BI用集計データ(nps_summary.csv)を保存しました。")

Power BI用集計データ(nps_summary.csv)を保存しました。
